In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 64
tree_depth = 10
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.198735237121582 | KNN Loss: 6.228330612182617 | BCE Loss: 1.9704046249389648
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.193399429321289 | KNN Loss: 6.228418827056885 | BCE Loss: 1.9649808406829834
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.223655700683594 | KNN Loss: 6.228407859802246 | BCE Loss: 1.9952478408813477
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.12940502166748 | KNN Loss: 6.228500843048096 | BCE Loss: 1.9009044170379639
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.163383483886719 | KNN Loss: 6.227762699127197 | BCE Loss: 1.9356210231781006
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.170571327209473 | KNN Loss: 6.227832317352295 | BCE Loss: 1.9427390098571777
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.164538383483887 | KNN Loss: 6.22742223739624 | BCE Loss: 1.9371163845062256
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.091979026794434 | KNN Loss: 6.22742223739624 | BCE Loss: 1.86455714

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 6.738680839538574 | KNN Loss: 5.639242649078369 | BCE Loss: 1.099438190460205
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 6.765195846557617 | KNN Loss: 5.623289108276367 | BCE Loss: 1.14190673828125
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 6.651707649230957 | KNN Loss: 5.5264081954956055 | BCE Loss: 1.1252994537353516
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 6.5760297775268555 | KNN Loss: 5.452567100524902 | BCE Loss: 1.1234629154205322
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 6.507997035980225 | KNN Loss: 5.396655082702637 | BCE Loss: 1.1113418340682983
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 6.445337295532227 | KNN Loss: 5.33567476272583 | BCE Loss: 1.1096622943878174
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 6.339080810546875 | KNN Loss: 5.23173189163208 | BCE Loss: 1.107349157333374
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 6.220019817352295 | KNN Loss: 5.13062858581543 | BCE Loss: 1.0

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 5.498337745666504 | KNN Loss: 4.450432777404785 | BCE Loss: 1.0479047298431396
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 5.499588966369629 | KNN Loss: 4.450000286102295 | BCE Loss: 1.049588918685913
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 5.5169782638549805 | KNN Loss: 4.461717128753662 | BCE Loss: 1.0552613735198975
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 5.492061614990234 | KNN Loss: 4.438493251800537 | BCE Loss: 1.0535683631896973
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 5.5384840965271 | KNN Loss: 4.464260101318359 | BCE Loss: 1.0742239952087402
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 5.467388153076172 | KNN Loss: 4.433053493499756 | BCE Loss: 1.0343348979949951
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 5.527736663818359 | KNN Loss: 4.471822261810303 | BCE Loss: 1.0559141635894775
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 5.500675201416016 | KNN Loss: 4.458330154418945 | BCE Loss:

Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 5.4654645919799805 | KNN Loss: 4.422767162322998 | BCE Loss: 1.0426971912384033
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 5.493514060974121 | KNN Loss: 4.415186405181885 | BCE Loss: 1.0783274173736572
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 5.479903221130371 | KNN Loss: 4.41825532913208 | BCE Loss: 1.0616480112075806
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 5.450873374938965 | KNN Loss: 4.432248592376709 | BCE Loss: 1.018625020980835
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 5.460605621337891 | KNN Loss: 4.403741359710693 | BCE Loss: 1.0568640232086182
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 5.501046180725098 | KNN Loss: 4.443245887756348 | BCE Loss: 1.05780029296875
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 5.489946365356445 | KNN Loss: 4.433122158050537 | BCE Loss: 1.0568244457244873
Epoch 33 / 500 | iteration 20 / 30 | Total Loss: 5.447597980499268 | KNN Loss: 4.405772686004639 | BCE Loss: 

Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 5.494498252868652 | KNN Loss: 4.442195892333984 | BCE Loss: 1.0523024797439575
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 5.465056896209717 | KNN Loss: 4.438093185424805 | BCE Loss: 1.0269638299942017
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 5.502208709716797 | KNN Loss: 4.44742488861084 | BCE Loss: 1.054783582687378
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 5.425683498382568 | KNN Loss: 4.4065117835998535 | BCE Loss: 1.0191717147827148
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 5.431161880493164 | KNN Loss: 4.394833564758301 | BCE Loss: 1.0363280773162842
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 5.4571638107299805 | KNN Loss: 4.421267509460449 | BCE Loss: 1.0358960628509521
Epoch 44 / 500 | iteration 10 / 30 | Total Loss: 5.449094295501709 | KNN Loss: 4.409332752227783 | BCE Loss: 1.0397614240646362
Epoch 44 / 500 | iteration 15 / 30 | Total Loss: 5.469348907470703 | KNN Loss: 4.410404205322266 | BCE Los

...

Epoch 449 / 500 | iteration 5 / 30 | Total Loss: 5.3634033203125 | KNN Loss: 4.333193778991699 | BCE Loss: 1.0302095413208008
Epoch 449 / 500 | iteration 10 / 30 | Total Loss: 5.404452800750732 | KNN Loss: 4.383240699768066 | BCE Loss: 1.0212122201919556
Epoch 449 / 500 | iteration 15 / 30 | Total Loss: 5.322530269622803 | KNN Loss: 4.326475143432617 | BCE Loss: 0.9960550665855408
Epoch 449 / 500 | iteration 20 / 30 | Total Loss: 5.410157203674316 | KNN Loss: 4.3770952224731445 | BCE Loss: 1.033062219619751
Epoch 449 / 500 | iteration 25 / 30 | Total Loss: 5.386936664581299 | KNN Loss: 4.351184844970703 | BCE Loss: 1.0357518196105957
Epoch 450 / 500 | iteration 0 / 30 | Total Loss: 5.3716936111450195 | KNN Loss: 4.333949089050293 | BCE Loss: 1.0377447605133057
Epoch 450 / 500 | iteration 5 / 30 | Total Loss: 5.333678722381592 | KNN Loss: 4.344855308532715 | BCE Loss: 0.9888232946395874
Epoch 450 / 500 | iteration 10 / 30 | Total Loss: 5.397200107574463 | KNN Loss: 4.3757147789001465 | 

Epoch 459 / 500 | iteration 25 / 30 | Total Loss: 5.342750549316406 | KNN Loss: 4.327213287353516 | BCE Loss: 1.0155372619628906
Epoch 460 / 500 | iteration 0 / 30 | Total Loss: 5.390583038330078 | KNN Loss: 4.345832824707031 | BCE Loss: 1.0447500944137573
Epoch 460 / 500 | iteration 5 / 30 | Total Loss: 5.360966205596924 | KNN Loss: 4.330828666687012 | BCE Loss: 1.030137538909912
Epoch 460 / 500 | iteration 10 / 30 | Total Loss: 5.360551834106445 | KNN Loss: 4.328232288360596 | BCE Loss: 1.0323193073272705
Epoch 460 / 500 | iteration 15 / 30 | Total Loss: 5.377540111541748 | KNN Loss: 4.339588165283203 | BCE Loss: 1.0379518270492554
Epoch 460 / 500 | iteration 20 / 30 | Total Loss: 5.392430782318115 | KNN Loss: 4.370214462280273 | BCE Loss: 1.0222163200378418
Epoch 460 / 500 | iteration 25 / 30 | Total Loss: 5.371382713317871 | KNN Loss: 4.358486652374268 | BCE Loss: 1.0128958225250244
Epoch 461 / 500 | iteration 0 / 30 | Total Loss: 5.382922172546387 | KNN Loss: 4.359556674957275 | B

Epoch 470 / 500 | iteration 15 / 30 | Total Loss: 5.358429431915283 | KNN Loss: 4.32494592666626 | BCE Loss: 1.0334833860397339
Epoch 470 / 500 | iteration 20 / 30 | Total Loss: 5.368271350860596 | KNN Loss: 4.356976509094238 | BCE Loss: 1.011294960975647
Epoch 470 / 500 | iteration 25 / 30 | Total Loss: 5.3690080642700195 | KNN Loss: 4.339217662811279 | BCE Loss: 1.0297901630401611
Epoch 471 / 500 | iteration 0 / 30 | Total Loss: 5.34496545791626 | KNN Loss: 4.337339878082275 | BCE Loss: 1.0076255798339844
Epoch 471 / 500 | iteration 5 / 30 | Total Loss: 5.389900207519531 | KNN Loss: 4.372283458709717 | BCE Loss: 1.0176165103912354
Epoch 471 / 500 | iteration 10 / 30 | Total Loss: 5.404962539672852 | KNN Loss: 4.372114658355713 | BCE Loss: 1.0328476428985596
Epoch 471 / 500 | iteration 15 / 30 | Total Loss: 5.380067825317383 | KNN Loss: 4.3546648025512695 | BCE Loss: 1.0254027843475342
Epoch 471 / 500 | iteration 20 / 30 | Total Loss: 5.427299499511719 | KNN Loss: 4.386161804199219 | 

Epoch 481 / 500 | iteration 5 / 30 | Total Loss: 5.369363784790039 | KNN Loss: 4.339537620544434 | BCE Loss: 1.0298261642456055
Epoch 481 / 500 | iteration 10 / 30 | Total Loss: 5.389568328857422 | KNN Loss: 4.364975929260254 | BCE Loss: 1.024592399597168
Epoch 481 / 500 | iteration 15 / 30 | Total Loss: 5.369410514831543 | KNN Loss: 4.359279155731201 | BCE Loss: 1.0101313591003418
Epoch 481 / 500 | iteration 20 / 30 | Total Loss: 5.3662590980529785 | KNN Loss: 4.34240198135376 | BCE Loss: 1.0238569974899292
Epoch 481 / 500 | iteration 25 / 30 | Total Loss: 5.434374809265137 | KNN Loss: 4.3694376945495605 | BCE Loss: 1.0649373531341553
Epoch 482 / 500 | iteration 0 / 30 | Total Loss: 5.386667728424072 | KNN Loss: 4.363371849060059 | BCE Loss: 1.0232957601547241
Epoch 482 / 500 | iteration 5 / 30 | Total Loss: 5.3647918701171875 | KNN Loss: 4.347118854522705 | BCE Loss: 1.0176732540130615
Epoch 482 / 500 | iteration 10 / 30 | Total Loss: 5.4127044677734375 | KNN Loss: 4.358959197998047 

Epoch 491 / 500 | iteration 25 / 30 | Total Loss: 5.353229522705078 | KNN Loss: 4.346161365509033 | BCE Loss: 1.007068395614624
Epoch 492 / 500 | iteration 0 / 30 | Total Loss: 5.421252250671387 | KNN Loss: 4.371200084686279 | BCE Loss: 1.0500521659851074
Epoch 492 / 500 | iteration 5 / 30 | Total Loss: 5.3846845626831055 | KNN Loss: 4.335709571838379 | BCE Loss: 1.0489747524261475
Epoch 492 / 500 | iteration 10 / 30 | Total Loss: 5.403069496154785 | KNN Loss: 4.333566665649414 | BCE Loss: 1.0695027112960815
Epoch 492 / 500 | iteration 15 / 30 | Total Loss: 5.36777400970459 | KNN Loss: 4.340906143188477 | BCE Loss: 1.0268676280975342
Epoch 492 / 500 | iteration 20 / 30 | Total Loss: 5.383893966674805 | KNN Loss: 4.3631415367126465 | BCE Loss: 1.020752191543579
Epoch 492 / 500 | iteration 25 / 30 | Total Loss: 5.348280429840088 | KNN Loss: 4.353436470031738 | BCE Loss: 0.9948439598083496
Epoch 493 / 500 | iteration 0 / 30 | Total Loss: 5.375085353851318 | KNN Loss: 4.338935852050781 | B

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 2.3926,  4.0860,  2.8788,  3.9504,  2.8986,  0.4837,  2.7811,  1.9030,
          2.1603,  2.3338,  2.5971,  2.5487,  1.0050,  1.4871,  1.5329,  1.5996,
          2.2310,  3.5485,  2.7080,  1.6907,  2.0699,  2.6769,  2.0625,  2.9899,
          1.9688,  2.0603,  2.1489,  1.4235,  1.8046,  0.2838, -0.4469,  0.9981,
         -0.2694,  0.8887,  1.6710,  1.5721,  0.9988,  3.7110,  1.0052,  1.3846,
          0.9644, -0.8202, -0.4330,  2.1562,  2.2686,  0.7811, -0.1593,  0.0440,
          1.2229,  2.8391,  2.1248,  0.1024,  1.5054,  0.5918, -0.5685,  1.0243,
          1.4711,  1.4002,  1.3616,  2.1332,  0.5040,  0.9411,  0.2120,  1.7772,
          1.1621,  1.6978, -2.1811,  0.0616,  2.3492,  2.2480,  1.9991,  0.3823,
          1.1766,  2.1853,  2.2963,  1.0241,  0.3581,  0.8016,  0.1199,  1.6242,
          0.0295,  0.4071,  1.8388, -0.4244,  0.2974, -1.0537, -2.5488, -0.3111,
          0.5336, -2.0137,  0.4038, -0.1622, -0.6053, -0.9296,  0.4635,  1.2992,
         -0.6913, -0.6401,  

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [10]:
dataset_ = [d[0].cpu() for d in dataset]

In [11]:
model = model.eval().cpu()
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 88.88it/s] 


In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [13]:
clusters = DBSCAN(eps=0.2, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [14]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [16]:
tensor_dataset = torch.stack(dataset_)

In [17]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [18]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [19]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [20]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [21]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [22]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [23]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [24]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [25]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [26]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [27]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [28]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [29]:
losses = []
accs = []
sparsity = []

In [30]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
layer 7: 0.0
layer 8: 0.0
Epoch: 00 | Batch: 000 / 029 | Total loss: 9.607 | Reg loss: 0.011 | Tree loss: 9.607 | Accuracy: 0.000000 | 1.088 sec/iter
Epoch: 00 | Batch: 001 / 029 | Total loss: 9.603 | Reg loss: 0.011 | Tree loss: 9.603 | Accuracy: 0.000000 | 0.95 sec/iter
Epoch: 00 | Batch: 002 / 029 | Total loss: 9.598 | Reg loss: 0.010 | Tree loss: 9.598 | Accuracy: 0.000000 | 0.909 sec/iter
Epoch: 00 | Batch: 003 / 029 | Total loss: 9.594 | Reg loss: 0.009 | Tree loss: 9.594 | Accuracy: 0.000000 | 0.892 sec/iter
Epoch: 00 | Batch: 004 / 029 | Total loss: 9.588 | Reg loss: 0.009 | Tree loss: 9.588 | Accuracy: 0.000000 | 0.88 sec/iter
Epoch: 00 | Batch: 005 / 029 | Total loss: 9.582 | Reg loss: 0.008 | Tree loss: 9.582 | Accuracy: 0.000000 | 0.874 sec/iter
Epoch: 00 | Batch: 006 / 029 | Total loss: 9.577 | Reg loss: 0.008 | Tree loss: 9.577 | Accuracy: 0.000000 | 0.868 se

Epoch: 02 | Batch: 003 / 029 | Total loss: 9.457 | Reg loss: 0.008 | Tree loss: 9.457 | Accuracy: 0.218750 | 0.863 sec/iter
Epoch: 02 | Batch: 004 / 029 | Total loss: 9.452 | Reg loss: 0.008 | Tree loss: 9.452 | Accuracy: 0.189453 | 0.863 sec/iter
Epoch: 02 | Batch: 005 / 029 | Total loss: 9.444 | Reg loss: 0.008 | Tree loss: 9.444 | Accuracy: 0.220703 | 0.862 sec/iter
Epoch: 02 | Batch: 006 / 029 | Total loss: 9.439 | Reg loss: 0.008 | Tree loss: 9.439 | Accuracy: 0.193359 | 0.861 sec/iter
Epoch: 02 | Batch: 007 / 029 | Total loss: 9.436 | Reg loss: 0.009 | Tree loss: 9.436 | Accuracy: 0.212891 | 0.861 sec/iter
Epoch: 02 | Batch: 008 / 029 | Total loss: 9.427 | Reg loss: 0.009 | Tree loss: 9.427 | Accuracy: 0.222656 | 0.86 sec/iter
Epoch: 02 | Batch: 009 / 029 | Total loss: 9.424 | Reg loss: 0.009 | Tree loss: 9.424 | Accuracy: 0.212891 | 0.86 sec/iter
Epoch: 02 | Batch: 010 / 029 | Total loss: 9.416 | Reg loss: 0.010 | Tree loss: 9.416 | Accuracy: 0.224609 | 0.86 sec/iter
Epoch: 02 |

Epoch: 04 | Batch: 007 / 029 | Total loss: 9.230 | Reg loss: 0.015 | Tree loss: 9.230 | Accuracy: 0.246094 | 0.86 sec/iter
Epoch: 04 | Batch: 008 / 029 | Total loss: 9.230 | Reg loss: 0.015 | Tree loss: 9.230 | Accuracy: 0.199219 | 0.86 sec/iter
Epoch: 04 | Batch: 009 / 029 | Total loss: 9.211 | Reg loss: 0.015 | Tree loss: 9.211 | Accuracy: 0.222656 | 0.86 sec/iter
Epoch: 04 | Batch: 010 / 029 | Total loss: 9.202 | Reg loss: 0.016 | Tree loss: 9.202 | Accuracy: 0.205078 | 0.86 sec/iter
Epoch: 04 | Batch: 011 / 029 | Total loss: 9.198 | Reg loss: 0.016 | Tree loss: 9.198 | Accuracy: 0.191406 | 0.86 sec/iter
Epoch: 04 | Batch: 012 / 029 | Total loss: 9.175 | Reg loss: 0.017 | Tree loss: 9.175 | Accuracy: 0.199219 | 0.86 sec/iter
Epoch: 04 | Batch: 013 / 029 | Total loss: 9.169 | Reg loss: 0.017 | Tree loss: 9.169 | Accuracy: 0.195312 | 0.859 sec/iter
Epoch: 04 | Batch: 014 / 029 | Total loss: 9.154 | Reg loss: 0.018 | Tree loss: 9.154 | Accuracy: 0.199219 | 0.859 sec/iter
Epoch: 04 | Ba

Epoch: 06 | Batch: 011 / 029 | Total loss: 8.789 | Reg loss: 0.022 | Tree loss: 8.789 | Accuracy: 0.220703 | 0.861 sec/iter
Epoch: 06 | Batch: 012 / 029 | Total loss: 8.775 | Reg loss: 0.022 | Tree loss: 8.775 | Accuracy: 0.212891 | 0.861 sec/iter
Epoch: 06 | Batch: 013 / 029 | Total loss: 8.758 | Reg loss: 0.022 | Tree loss: 8.758 | Accuracy: 0.193359 | 0.861 sec/iter
Epoch: 06 | Batch: 014 / 029 | Total loss: 8.708 | Reg loss: 0.023 | Tree loss: 8.708 | Accuracy: 0.216797 | 0.861 sec/iter
Epoch: 06 | Batch: 015 / 029 | Total loss: 8.712 | Reg loss: 0.023 | Tree loss: 8.712 | Accuracy: 0.187500 | 0.86 sec/iter
Epoch: 06 | Batch: 016 / 029 | Total loss: 8.705 | Reg loss: 0.024 | Tree loss: 8.705 | Accuracy: 0.201172 | 0.86 sec/iter
Epoch: 06 | Batch: 017 / 029 | Total loss: 8.679 | Reg loss: 0.024 | Tree loss: 8.679 | Accuracy: 0.191406 | 0.86 sec/iter
Epoch: 06 | Batch: 018 / 029 | Total loss: 8.624 | Reg loss: 0.024 | Tree loss: 8.624 | Accuracy: 0.201172 | 0.86 sec/iter
Epoch: 06 | 

Epoch: 08 | Batch: 015 / 029 | Total loss: 8.196 | Reg loss: 0.027 | Tree loss: 8.196 | Accuracy: 0.181641 | 0.864 sec/iter
Epoch: 08 | Batch: 016 / 029 | Total loss: 8.162 | Reg loss: 0.027 | Tree loss: 8.162 | Accuracy: 0.189453 | 0.864 sec/iter
Epoch: 08 | Batch: 017 / 029 | Total loss: 8.122 | Reg loss: 0.027 | Tree loss: 8.122 | Accuracy: 0.230469 | 0.864 sec/iter
Epoch: 08 | Batch: 018 / 029 | Total loss: 8.113 | Reg loss: 0.028 | Tree loss: 8.113 | Accuracy: 0.167969 | 0.863 sec/iter
Epoch: 08 | Batch: 019 / 029 | Total loss: 8.111 | Reg loss: 0.028 | Tree loss: 8.111 | Accuracy: 0.212891 | 0.863 sec/iter
Epoch: 08 | Batch: 020 / 029 | Total loss: 8.069 | Reg loss: 0.028 | Tree loss: 8.069 | Accuracy: 0.207031 | 0.863 sec/iter
Epoch: 08 | Batch: 021 / 029 | Total loss: 8.053 | Reg loss: 0.028 | Tree loss: 8.053 | Accuracy: 0.214844 | 0.863 sec/iter
Epoch: 08 | Batch: 022 / 029 | Total loss: 8.055 | Reg loss: 0.029 | Tree loss: 8.055 | Accuracy: 0.201172 | 0.863 sec/iter
Epoch: 0

...

Epoch: 89 | Batch: 013 / 029 | Total loss: 2.228 | Reg loss: 0.030 | Tree loss: 2.228 | Accuracy: 0.205078 | 0.836 sec/iter
Epoch: 89 | Batch: 014 / 029 | Total loss: 2.143 | Reg loss: 0.030 | Tree loss: 2.143 | Accuracy: 0.199219 | 0.836 sec/iter
Epoch: 89 | Batch: 015 / 029 | Total loss: 2.180 | Reg loss: 0.030 | Tree loss: 2.180 | Accuracy: 0.222656 | 0.836 sec/iter
Epoch: 89 | Batch: 016 / 029 | Total loss: 2.165 | Reg loss: 0.030 | Tree loss: 2.165 | Accuracy: 0.197266 | 0.836 sec/iter
Epoch: 89 | Batch: 017 / 029 | Total loss: 2.177 | Reg loss: 0.030 | Tree loss: 2.177 | Accuracy: 0.189453 | 0.836 sec/iter
Epoch: 89 | Batch: 018 / 029 | Total loss: 2.228 | Reg loss: 0.030 | Tree loss: 2.228 | Accuracy: 0.177734 | 0.836 sec/iter
Epoch: 89 | Batch: 019 / 029 | Total loss: 2.139 | Reg loss: 0.030 | Tree loss: 2.139 | Accuracy: 0.191406 | 0.836 sec/iter
Epoch: 89 | Batch: 020 / 029 | Total loss: 2.177 | Reg loss: 0.030 | Tree loss: 2.177 | Accuracy: 0.210938 | 0.836 sec/iter
Epoch: 8

Epoch: 91 | Batch: 017 / 029 | Total loss: 2.161 | Reg loss: 0.030 | Tree loss: 2.161 | Accuracy: 0.214844 | 0.834 sec/iter
Epoch: 91 | Batch: 018 / 029 | Total loss: 2.129 | Reg loss: 0.030 | Tree loss: 2.129 | Accuracy: 0.208984 | 0.834 sec/iter
Epoch: 91 | Batch: 019 / 029 | Total loss: 2.124 | Reg loss: 0.030 | Tree loss: 2.124 | Accuracy: 0.244141 | 0.834 sec/iter
Epoch: 91 | Batch: 020 / 029 | Total loss: 2.139 | Reg loss: 0.030 | Tree loss: 2.139 | Accuracy: 0.207031 | 0.834 sec/iter
Epoch: 91 | Batch: 021 / 029 | Total loss: 2.152 | Reg loss: 0.030 | Tree loss: 2.152 | Accuracy: 0.193359 | 0.834 sec/iter
Epoch: 91 | Batch: 022 / 029 | Total loss: 2.175 | Reg loss: 0.030 | Tree loss: 2.175 | Accuracy: 0.189453 | 0.834 sec/iter
Epoch: 91 | Batch: 023 / 029 | Total loss: 2.136 | Reg loss: 0.030 | Tree loss: 2.136 | Accuracy: 0.208984 | 0.834 sec/iter
Epoch: 91 | Batch: 024 / 029 | Total loss: 2.181 | Reg loss: 0.030 | Tree loss: 2.181 | Accuracy: 0.201172 | 0.834 sec/iter
Epoch: 9

Epoch: 93 | Batch: 021 / 029 | Total loss: 2.144 | Reg loss: 0.030 | Tree loss: 2.144 | Accuracy: 0.212891 | 0.833 sec/iter
Epoch: 93 | Batch: 022 / 029 | Total loss: 2.130 | Reg loss: 0.030 | Tree loss: 2.130 | Accuracy: 0.244141 | 0.833 sec/iter
Epoch: 93 | Batch: 023 / 029 | Total loss: 2.118 | Reg loss: 0.030 | Tree loss: 2.118 | Accuracy: 0.224609 | 0.833 sec/iter
Epoch: 93 | Batch: 024 / 029 | Total loss: 2.154 | Reg loss: 0.030 | Tree loss: 2.154 | Accuracy: 0.179688 | 0.833 sec/iter
Epoch: 93 | Batch: 025 / 029 | Total loss: 2.112 | Reg loss: 0.030 | Tree loss: 2.112 | Accuracy: 0.214844 | 0.833 sec/iter
Epoch: 93 | Batch: 026 / 029 | Total loss: 2.175 | Reg loss: 0.030 | Tree loss: 2.175 | Accuracy: 0.199219 | 0.832 sec/iter
Epoch: 93 | Batch: 027 / 029 | Total loss: 2.179 | Reg loss: 0.030 | Tree loss: 2.179 | Accuracy: 0.185547 | 0.832 sec/iter
Epoch: 93 | Batch: 028 / 029 | Total loss: 2.145 | Reg loss: 0.030 | Tree loss: 2.145 | Accuracy: 0.206897 | 0.832 sec/iter
Average 

Epoch: 95 | Batch: 025 / 029 | Total loss: 2.148 | Reg loss: 0.030 | Tree loss: 2.148 | Accuracy: 0.179688 | 0.831 sec/iter
Epoch: 95 | Batch: 026 / 029 | Total loss: 2.130 | Reg loss: 0.030 | Tree loss: 2.130 | Accuracy: 0.228516 | 0.831 sec/iter
Epoch: 95 | Batch: 027 / 029 | Total loss: 2.137 | Reg loss: 0.030 | Tree loss: 2.137 | Accuracy: 0.189453 | 0.831 sec/iter
Epoch: 95 | Batch: 028 / 029 | Total loss: 2.106 | Reg loss: 0.030 | Tree loss: 2.106 | Accuracy: 0.206897 | 0.831 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 96 | Batch: 000 / 029 | Total loss: 2.241 | Reg loss: 0.029 | Tree loss: 2.241 | Accuracy: 0.205078 | 0.831 sec/iter
Epoch: 96 | Batch: 001 / 029 | Total loss: 2.257 | Reg loss: 0.029 | Tree loss: 2.257 | Ac

Epoch: 98 | Batch: 000 / 029 | Total loss: 2.238 | Reg loss: 0.029 | Tree loss: 2.238 | Accuracy: 0.210938 | 0.829 sec/iter
Epoch: 98 | Batch: 001 / 029 | Total loss: 2.290 | Reg loss: 0.029 | Tree loss: 2.290 | Accuracy: 0.177734 | 0.829 sec/iter
Epoch: 98 | Batch: 002 / 029 | Total loss: 2.235 | Reg loss: 0.029 | Tree loss: 2.235 | Accuracy: 0.199219 | 0.829 sec/iter
Epoch: 98 | Batch: 003 / 029 | Total loss: 2.235 | Reg loss: 0.029 | Tree loss: 2.235 | Accuracy: 0.226562 | 0.829 sec/iter
Epoch: 98 | Batch: 004 / 029 | Total loss: 2.217 | Reg loss: 0.029 | Tree loss: 2.217 | Accuracy: 0.199219 | 0.829 sec/iter
Epoch: 98 | Batch: 005 / 029 | Total loss: 2.212 | Reg loss: 0.029 | Tree loss: 2.212 | Accuracy: 0.214844 | 0.829 sec/iter
Epoch: 98 | Batch: 006 / 029 | Total loss: 2.232 | Reg loss: 0.029 | Tree loss: 2.232 | Accuracy: 0.195312 | 0.829 sec/iter
Epoch: 98 | Batch: 007 / 029 | Total loss: 2.207 | Reg loss: 0.029 | Tree loss: 2.207 | Accuracy: 0.208984 | 0.828 sec/iter
Epoch: 9

In [31]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [33]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 9.895196506550219


# Extract Rules

# Accumulate samples in the leaves

In [34]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 916


In [35]:
method = 'greedy'

In [36]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [37]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))


============== Pattern 1 ==============
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
============== Pattern 25

============== Pattern 659 ==============
============== Pattern 660 ==============
============== Pattern 661 ==============
============== Pattern 662 ==============
============== Pattern 663 ==============
============== Pattern 664 ==============
============== Pattern 665 ==============
============== Pattern 666 ==============
============== Pattern 667 ==============
============== Pattern 668 ==============
============== Pattern 669 ==============
============== Pattern 670 ==============
============== Pattern 671 ==============
============== Pattern 672 ==============
============== Pattern 673 ==============
============== Pattern 674 ==============
============== Pattern 675 ==============
============== Pattern 676 ==============
============== Pattern 677 ==============
============== Pattern 678 ==============
============== Pattern 679 ==============
============== Pattern 680 ==============
============== Pattern 681 ==============
============== Pattern 682 =======